In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

df = pd.read_csv(r'./datasets/game_profile.csv')

# Create a TF-IDF Vectorizer to transform the 'profile' into numerical data
tfidf = TfidfVectorizer(stop_words='english',max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['profile'])

# Use k-NN to find similar games
knn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')  # Using 6 neighbors to include the game itself
knn.fit(tfidf_matrix)



NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [3]:
# Crear funcion que crea las recomendaciones
def recommend(game_id):
    try:
        idx = df.index[df['id'] == game_id].tolist()[0]
        name = df['title'].loc[idx]
        # print(f'Porque te gusto "{name}" te recomendamos:')
        values , indices = knn.kneighbors(tfidf_matrix[idx])
        
        # Exclude the first (index 0) since it'll be the game itself
        game_indices = indices[0][1:]
        result = df['id'].iloc[game_indices]
    except:
        return [0]
    return list(result)

In [6]:
recommend(47810)

[47730.0, 17450.0, 17460.0, 7110.0, 24980.0]

In [8]:
# un df sin el profile
df1 = df[['id','title']]

In [10]:

df1['recommends'] = df1['id'].apply(lambda x  : recommend(x))


C:\Users\octav\AppData\Local\Temp\ipykernel_16408\2150983915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['recommends'] = df1['id'].apply(lambda x  : recommend(x))


In [11]:
df1['id'].fillna(0,inplace=True)
df1.to_csv(r'./dataquery/model_item.csv',index=False)
df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30085 entries, 0 to 30084
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          30085 non-null  float64
 1   title       30085 non-null  object 
 2   recommends  30085 non-null  object 
dtypes: float64(1), object(2)
memory usage: 705.2+ KB


C:\Users\octav\AppData\Local\Temp\ipykernel_16408\2525660285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['id'].fillna(0,inplace=True)


In [26]:
import ast

# Función del endpoint
def recomendacion_juego( game_id : int ):
    # carga de archivos
    df = pd.read_csv(r'./dataquery/model_item.csv')

    if game_id not in df['id'].values:
        return 'error'

    # Obtiene la lista de recomendaciones
    result = df[df['id'] == game_id]['recommends'].iloc[0]

    try:
        result = ast.literal_eval(result)
    except (SyntaxError, ValueError):
        print('No se puede convertir a lista')
        return 'error'

    response =[]
    for item_id in result:
        # Obtiene la información del juego recomendado
        item_info = df[df['id'] == item_id].iloc[0]  

        response.append({'game_id': int(item_info['id']), 
                         'title': item_info['title']})

    return response


In [27]:
recomendacion_juego(47810)

[{'game_id': 47730, 'title': 'Dragon Age™: Origins Awakening'},
 {'game_id': 17450, 'title': 'Dragon Age: Origins'},
 {'game_id': 17460, 'title': 'Mass Effect'},
 {'game_id': 7110, 'title': 'Jade Empire™: Special Edition'},
 {'game_id': 24980, 'title': 'Mass Effect 2'}]